# HTMXSSEConnector

> HTMX-specific SSE integration helpers for FastHTML. Simplifies adding SSE attributes, creating SSE-enabled elements, and managing HTMX SSE connections.

In [ ]:
#| default_exp htmx

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from typing import Optional, Dict, Any, Union, List
from fasthtml.common import Div

In [ ]:
#| export
class HTMXSSEConnector:
    """
    Provides helper functions for setting up HTMX SSE connections
    without hardcoding specific implementations.
    """
    
    @staticmethod
    def add_sse_attrs(element, 
                      endpoint: str,  # SSE endpoint URL to connect to
                      events: Optional[Union[str, List[str]]] = None,
                      swap_type: str = "message",  # How to swap content (message, innerHTML, outerHTML, etc.)
                      auto_reconnect: bool = True):
        """
        Add SSE connection attributes to an element.
        
        Args:
            element: The element to add SSE attributes to
            endpoint: SSE endpoint URL
            events: Optional event name(s) to listen for
            swap_type: How to swap content (message, innerHTML, outerHTML, etc.)
            auto_reconnect: Whether to auto-reconnect on disconnect
            
        Returns:
            The element with SSE attributes added
        """
        if hasattr(element, 'attrs'):
            element.attrs['hx-ext'] = 'sse'
            element.attrs['sse-connect'] = endpoint
            
            if events:
                if isinstance(events, list):
                    for event in events:
                        element.attrs[f'sse-swap'] = event
                else:
                    element.attrs['sse-swap'] = events
            else:
                element.attrs['sse-swap'] = swap_type
                
            if not auto_reconnect:
                element.attrs['sse-close'] = 'true'
        
        return element
    
    @staticmethod
    def create_sse_element(element_type=Div,
                          endpoint: str = None,  # SSE endpoint URL to connect to
                          element_id: str = None,  # Optional ID for the element
                          events: Optional[Union[str, List[str]]] = None,
                          swap_type: str = "message",  # How to swap content when messages are received
                          hidden: bool = False,  # Whether to hide the element initially
                          **kwargs):
        """
        Create an element with SSE connection configured.
        
        Args:
            element_type: Type of element to create (Div, Span, etc.)
            endpoint: SSE endpoint URL
            element_id: Optional element ID
            events: Optional event name(s) to listen for
            swap_type: How to swap content
            hidden: Whether to hide the element
            **kwargs: Additional attributes for the element
            
        Returns:
            Element configured for SSE connection
        """
        attrs = kwargs.copy()
        
        if element_id:
            attrs['id'] = element_id
            
        if endpoint:
            attrs['hx_ext'] = 'sse'
            attrs['sse_connect'] = endpoint
            
            if events:
                if isinstance(events, list):
                    # For multiple events, would need special handling
                    attrs['sse_swap'] = events[0] if events else swap_type
                else:
                    attrs['sse_swap'] = events
            else:
                attrs['sse_swap'] = swap_type
        
        if hidden:
            attrs['style'] = attrs.get('style', '') + 'display: none;'
        
        return element_type(**attrs)
    
    @staticmethod
    def sse_progress_element(job_id: str, 
                            endpoint_template: str = "/stream_job_progress?job_id={job_id}",  # URL template for the SSE endpoint
                            element_id_template: str = "progress-span-{job_id}",  # Template for generating element ID
                            initial_content=None):
        """
        Create an SSE-enabled progress element.
        
        Args:
            job_id: Job identifier
            endpoint_template: Template for SSE endpoint URL
            element_id_template: Template for element ID
            initial_content: Initial content to display
            
        Returns:
            SSE-configured element for progress updates
        """
        return Span(
            initial_content or "",
            id=element_id_template.format(job_id=job_id),
            hx_ext="sse",
            sse_connect=endpoint_template.format(job_id=job_id),
            sse_swap="message"
        )
    
    @staticmethod
    def sse_status_element(job_id: str,
                          endpoint_template: str = "/stream_job_status?job_id={job_id}",  # URL template for the SSE endpoint
                          element_id_template: str = "status-span-{job_id}",  # Template for generating element ID
                          initial_content=None):
        """
        Create an SSE-enabled status element.
        
        Args:
            job_id: Job identifier
            endpoint_template: Template for SSE endpoint URL
            element_id_template: Template for element ID
            initial_content: Initial content to display
            
        Returns:
            SSE-configured element for status updates
        """
        return Span(
            initial_content or "",
            id=element_id_template.format(job_id=job_id),
            hx_ext="sse",
            sse_connect=endpoint_template.format(job_id=job_id),
            sse_swap="message"
        )
    
    @staticmethod
    def create_sse_monitor_script(
        config: Dict[str, Any]  # Configuration dictionary for monitoring setup
    ):
        """
        Create a monitoring script for SSE connections.
        
        Args:
            config: Configuration dictionary with keys:
                - sse_element_id: ID of SSE element to monitor
                - status_element_id: ID of status display element
                - auto_reconnect: Whether to auto-reconnect
                - debug: Whether to enable debug logging
                - status_indicators: Dict of status HTML strings
                
        Returns:
            Script element with monitoring code
        """
        sse_id = config.get('sse_element_id', 'sse-connection')
        status_id = config.get('status_element_id', 'connection-status')
        auto_reconnect = config.get('auto_reconnect', True)
        debug = config.get('debug', False)
        indicators = config.get('status_indicators', {})
        
        # Default indicators if not provided
        if not indicators:
            indicators = {
                'active': '<span style="color: green;">Connected</span>',
                'disconnected': '<span style="color: red;">Disconnected</span>',
                'error': '<span style="color: orange;">Error</span>',
                'reconnecting': '<span style="color: blue;">Reconnecting...</span>'
            }
        
        js_code = f"""
        (function() {{
            const sseElementId = '{sse_id}';
            const statusElementId = '{status_id}';
            const debugLogging = {str(debug).lower()};
            const autoReconnect = {str(auto_reconnect).lower()};
            
            const statusIndicators = {json.dumps(indicators)};
            
            function log(message) {{
                if (debugLogging) {{
                    console.log('[SSE Monitor] ' + message);
                }}
            }}
            
            function updateStatus(status) {{
                const statusElement = document.getElementById(statusElementId);
                if (statusElement && statusIndicators[status]) {{
                    statusElement.innerHTML = statusIndicators[status];
                    log('Status updated to: ' + status);
                }}
            }}
            
            document.addEventListener('DOMContentLoaded', function() {{
                const sseElement = document.getElementById(sseElementId);
                
                if (!sseElement) {{
                    log('SSE element not found: ' + sseElementId);
                    return;
                }}
                
                // Monitor SSE connection events
                sseElement.addEventListener('htmx:sseOpen', function(evt) {{
                    log('SSE connection established');
                    updateStatus('active');
                }});
                
                sseElement.addEventListener('htmx:sseClose', function(evt) {{
                    log('SSE connection closed');
                    updateStatus('disconnected');
                }});
                
                sseElement.addEventListener('htmx:sseError', function(evt) {{
                    log('SSE connection error');
                    updateStatus('error');
                    
                    if (autoReconnect) {{
                        setTimeout(function() {{
                            updateStatus('reconnecting');
                            htmx.trigger(sseElement, 'htmx:sseReconnect');
                        }}, 3000);
                    }}
                }});
                
                // Handle tab visibility changes
                if (autoReconnect) {{
                    document.addEventListener('visibilitychange', function() {{
                        if (!document.hidden) {{
                            const sseData = sseElement['htmx-internal-data'];
                            
                            if (sseData && sseData.sseEventSource) {{
                                const state = sseData.sseEventSource.readyState;
                                
                                if (state === EventSource.CLOSED) {{
                                    log('Tab visible - reconnecting closed SSE');
                                    updateStatus('reconnecting');
                                    htmx.trigger(sseElement, 'htmx:sseReconnect');
                                }} else if (state === EventSource.OPEN) {{
                                    log('Tab visible - SSE already connected');
                                    updateStatus('active');
                                }} else if (state === EventSource.CONNECTING) {{
                                    log('Tab visible - SSE connecting');
                                    updateStatus('reconnecting');
                                }}
                            }}
                        }} else {{
                            log('Tab hidden');
                        }}
                    }});
                }}
                
                // Initial status check
                setTimeout(function() {{
                    const sseData = sseElement['htmx-internal-data'];
                    if (sseData && sseData.sseEventSource) {{
                        const state = sseData.sseEventSource.readyState;
                        if (state === EventSource.OPEN) {{
                            updateStatus('active');
                        }} else if (state === EventSource.CONNECTING) {{
                            updateStatus('reconnecting');
                        }} else {{
                            updateStatus('disconnected');
                        }}
                    }}
                }}, 100);
            }});
        }})();
        """
        
        return Script(js_code)

In [ ]:
# Initialize the HTMX SSE Connector
htmx_sse = HTMXSSEConnector()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()